# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))
    all_train_edges = adj_train.nonzero()
    out_train_edges = [[all_train_edges[0][i],all_train_edges[1][i]] for i in range(all_train_edges[0].shape[0])]
    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return np.array(out_train_edges), val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    train_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*4, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

# 定义模型

In [6]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        #return torch.exp(-nodes_similar)
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        #return torch.exp(-nodes_similar)

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [7]:
context_embedding_dim = 12
importance_embedding_dim = 48
beta = 4.2
ALPHA = 1.0
LR = 0.05
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'cora'

In [8]:
all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()

    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(50):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 4909.2588
Epoch: 01, Loss: 4846.0508
Epoch: 02, Loss: 4746.0298
Epoch: 03, Loss: 4645.0684
Epoch: 04, Loss: 4540.1406
AP SCORE：  0.55606474984974
AUC SCORE:  0.5412481908688216
Epoch: 00, Loss: 4421.3467
Epoch: 01, Loss: 4415.6406
Epoch: 02, Loss: 4405.0806
Epoch: 03, Loss: 4391.2871
Epoch: 04, Loss: 4378.0381
AP SCORE：  0.5962460182370432
AUC SCORE:  0.5958510591640718
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4371.2070
Epoch: 01, Loss: 4307.1050
Epoch: 02, Loss: 4236.4043
Epoch: 03, Loss: 4165.6079
Epoch: 04, Loss: 4068.7961
AP SCORE：  0.6648341386913525
AUC SCORE:  0.6634358142186747
Epoch: 00, Loss: 3960.3113
Epoch: 01, Loss: 3954.1138
Epoch: 02, Loss: 3948.4768
Epoch: 03, Loss: 3945.7366
Epoch: 04, Loss: 3944.9619
AP SCORE：  0.6938955789012198
AUC SCORE:  0.6950572343318275
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 3944.6147
Epoch

Epoch: 00, Loss: 2858.3770
Epoch: 01, Loss: 2856.3491
Epoch: 02, Loss: 2852.5181
Epoch: 03, Loss: 2847.1006
Epoch: 04, Loss: 2840.3035
AP SCORE：  0.7692813178179206
AUC SCORE:  0.7868953116091398
Epoch: 00, Loss: 2832.3196
Epoch: 01, Loss: 2832.2468
Epoch: 02, Loss: 2832.1252
Epoch: 03, Loss: 2831.9878
Epoch: 04, Loss: 2831.8628
AP SCORE：  0.7686484772856668
AUC SCORE:  0.7867637384325249
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 2831.7686
Epoch: 01, Loss: 2829.7954
Epoch: 02, Loss: 2826.0698
Epoch: 03, Loss: 2820.8118
Epoch: 04, Loss: 2814.2344
AP SCORE：  0.7696177705474548
AUC SCORE:  0.7865005920792948
Epoch: 00, Loss: 2806.5383
Epoch: 01, Loss: 2806.4744
Epoch: 02, Loss: 2806.3689
Epoch: 03, Loss: 2806.2512
Epoch: 04, Loss: 2806.1448
AP SCORE：  0.7689343716354754
AUC SCORE:  0.7863690189026797
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 2806.0642
Epoch: 01, Loss: 2804.1826
Epoch: 02, Loss: 2800.6331
Epoch: 03

Epoch: 01, Loss: 2498.2288
Epoch: 02, Loss: 2496.0215
Epoch: 03, Loss: 2492.8960
Epoch: 04, Loss: 2488.9680
AP SCORE：  0.7700181601083507
AUC SCORE:  0.7869830270602166
Epoch: 00, Loss: 2484.3408
Epoch: 01, Loss: 2484.3093
Epoch: 02, Loss: 2484.2605
Epoch: 03, Loss: 2484.2134
Epoch: 04, Loss: 2484.1760
AP SCORE：  0.7706486254896029
AUC SCORE:  0.7869391693346782
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 2484.1470
Epoch: 01, Loss: 2482.9934
Epoch: 02, Loss: 2480.8103
Epoch: 03, Loss: 2477.7163
Epoch: 04, Loss: 2473.8215
AP SCORE：  0.771534533820575
AUC SCORE:  0.7873338888645235
Epoch: 00, Loss: 2469.2273
Epoch: 01, Loss: 2469.1848
Epoch: 02, Loss: 2469.1174
Epoch: 03, Loss: 2469.0461
Epoch: 04, Loss: 2468.9812
AP SCORE：  0.7712260913980767
AUC SCORE:  0.7864567343537564
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 2468.9219
Epoch: 01, Loss: 2467.7673
Epoch: 02, Loss: 2465.5835
Epoch: 03, Loss: 2462.4937
Epoch: 04,

Epoch: 02, Loss: 3537.2061
Epoch: 03, Loss: 3515.2610
Epoch: 04, Loss: 3488.5486
AP SCORE：  0.6879866665044256
AUC SCORE:  0.7099250032893294
Epoch: 00, Loss: 3458.2517
Epoch: 01, Loss: 3458.0046
Epoch: 02, Loss: 3457.5564
Epoch: 03, Loss: 3456.9651
Epoch: 04, Loss: 3456.2854
AP SCORE：  0.6906454681275392
AUC SCORE:  0.7118547432130169
######################### 模块循环 ： 5 ##########################
Epoch: 00, Loss: 3455.5632
Epoch: 01, Loss: 3448.7795
Epoch: 02, Loss: 3436.1011
Epoch: 03, Loss: 3418.4746
Epoch: 04, Loss: 3396.8369
AP SCORE：  0.6954471521409296
AUC SCORE:  0.7150563571773167
Epoch: 00, Loss: 3372.0774
Epoch: 01, Loss: 3371.8655
Epoch: 02, Loss: 3371.4800
Epoch: 03, Loss: 3370.9634
Epoch: 04, Loss: 3370.3599
AP SCORE：  0.6980474515046442
AUC SCORE:  0.7173808166308495
######################### 模块循环 ： 6 ##########################
Epoch: 00, Loss: 3369.7085
Epoch: 01, Loss: 3364.0486
Epoch: 02, Loss: 3353.4443
Epoch: 03, Loss: 3338.6492
Epoch: 04, Loss: 3320.4109
AP SCORE：  

Epoch: 03, Loss: 2755.9158
Epoch: 04, Loss: 2750.4473
AP SCORE：  0.7204822070623537
AUC SCORE:  0.7362834963378799
Epoch: 00, Loss: 2744.0212
Epoch: 01, Loss: 2743.9807
Epoch: 02, Loss: 2743.9153
Epoch: 03, Loss: 2743.8411
Epoch: 04, Loss: 2743.7712
AP SCORE：  0.72037415199154
AUC SCORE:  0.7364589272400333
######################### 模块循环 ： 23 ##########################
Epoch: 00, Loss: 2743.7092
Epoch: 01, Loss: 2742.1409
Epoch: 02, Loss: 2739.1748
Epoch: 03, Loss: 2734.9773
Epoch: 04, Loss: 2729.7031
AP SCORE：  0.7201672955389351
AUC SCORE:  0.7369413622209552
Epoch: 00, Loss: 2723.4976
Epoch: 01, Loss: 2723.4570
Epoch: 02, Loss: 2723.3892
Epoch: 03, Loss: 2723.3101
Epoch: 04, Loss: 2723.2312
AP SCORE：  0.7198105263132097
AUC SCORE:  0.7372922240252621
######################### 模块循环 ： 24 ##########################
Epoch: 00, Loss: 2723.1567
Epoch: 01, Loss: 2721.6353
Epoch: 02, Loss: 2718.7559
Epoch: 03, Loss: 2714.6782
Epoch: 04, Loss: 2709.5505
AP SCORE：  0.719837041584683
AUC SCORE

...

Epoch: 02, Loss: 2922.5000
Epoch: 03, Loss: 2916.8179
Epoch: 04, Loss: 2909.6960
AP SCORE：  0.7362592523472987
AUC SCORE:  0.7381255208104908
Epoch: 00, Loss: 2901.3420
Epoch: 01, Loss: 2901.2844
Epoch: 02, Loss: 2901.1846
Epoch: 03, Loss: 2901.0603
Epoch: 04, Loss: 2900.9275
AP SCORE：  0.736878019272608
AUC SCORE:  0.7384763826147976
######################### 模块循环 ： 17 ##########################
Epoch: 00, Loss: 2900.7998
Epoch: 01, Loss: 2898.7764
Epoch: 02, Loss: 2894.9534
Epoch: 03, Loss: 2889.5479
Epoch: 04, Loss: 2882.7695
AP SCORE：  0.7378476201983667
AUC SCORE:  0.74001140300864
Epoch: 00, Loss: 2874.8145
Epoch: 01, Loss: 2874.7573
Epoch: 02, Loss: 2874.6575
Epoch: 03, Loss: 2874.5349
Epoch: 04, Loss: 2874.4060
AP SCORE：  0.7387923586328523
AUC SCORE:  0.7400552607341784
######################### 模块循环 ： 18 ##########################
Epoch: 00, Loss: 2874.2830
Epoch: 01, Loss: 2872.3569
Epoch: 02, Loss: 2868.7197
Epoch: 03, Loss: 2863.5791
Epoch: 04, Loss: 2857.1355
AP SCORE：  0

Epoch: 03, Loss: 2554.0608
Epoch: 04, Loss: 2550.0623
AP SCORE：  0.7457916846559743
AUC SCORE:  0.7538265865532213
Epoch: 00, Loss: 2545.3584
Epoch: 01, Loss: 2545.3228
Epoch: 02, Loss: 2545.2695
Epoch: 03, Loss: 2545.2166
Epoch: 04, Loss: 2545.1726
AP SCORE：  0.7446015759427808
AUC SCORE:  0.7536511556510679
######################### 模块循环 ： 35 ##########################
Epoch: 00, Loss: 2545.1362
Epoch: 01, Loss: 2543.9734
Epoch: 02, Loss: 2541.7761
Epoch: 03, Loss: 2538.6677
Epoch: 04, Loss: 2534.7627
AP SCORE：  0.7435512756894456
AUC SCORE:  0.7543090215341433
Epoch: 00, Loss: 2530.1650
Epoch: 01, Loss: 2530.1301
Epoch: 02, Loss: 2530.0776
Epoch: 03, Loss: 2530.0261
Epoch: 04, Loss: 2529.9836
AP SCORE：  0.7442742090830233
AUC SCORE:  0.7548791719661418
######################### 模块循环 ： 36 ##########################
Epoch: 00, Loss: 2529.9487
Epoch: 01, Loss: 2528.8074
Epoch: 02, Loss: 2526.6506
Epoch: 03, Loss: 2523.5957
Epoch: 04, Loss: 2519.7520
AP SCORE：  0.7437126686901501
AUC SC

Epoch: 04, Loss: 3819.0923
AP SCORE：  0.698049803723856
AUC SCORE:  0.7095302837594843
Epoch: 00, Loss: 3758.5535
Epoch: 01, Loss: 3757.3633
Epoch: 02, Loss: 3755.9092
Epoch: 03, Loss: 3754.7039
Epoch: 04, Loss: 3753.6135
AP SCORE：  0.7032886588142702
AUC SCORE:  0.718564975220385
######################### 模块循环 ： 3 ##########################
Epoch: 00, Loss: 3752.3911
Epoch: 01, Loss: 3739.5723
Epoch: 02, Loss: 3716.0862
Epoch: 03, Loss: 3684.3694
Epoch: 04, Loss: 3646.8499
AP SCORE：  0.7083239054905954
AUC SCORE:  0.7290469716240515
Epoch: 00, Loss: 3605.6763
Epoch: 01, Loss: 3605.3274
Epoch: 02, Loss: 3604.7222
Epoch: 03, Loss: 3603.9580
Epoch: 04, Loss: 3603.1091
AP SCORE：  0.7107811373217489
AUC SCORE:  0.7346607604929608
######################### 模块循环 ： 4 ##########################
Epoch: 00, Loss: 3602.2236
Epoch: 01, Loss: 3593.3428
Epoch: 02, Loss: 3576.8457
Epoch: 03, Loss: 3554.1240
Epoch: 04, Loss: 3526.5640
AP SCORE：  0.7134545566950758
AUC SCORE:  0.7410201306960221
Epoch:

Epoch: 00, Loss: 2814.9932
Epoch: 01, Loss: 2814.9482
Epoch: 02, Loss: 2814.8755
Epoch: 03, Loss: 2814.7949
Epoch: 04, Loss: 2814.7214
AP SCORE：  0.7407383889955977
AUC SCORE:  0.7643962984079646
######################### 模块循环 ： 21 ##########################
Epoch: 00, Loss: 2814.6589
Epoch: 01, Loss: 2812.9238
Epoch: 02, Loss: 2809.6443
Epoch: 03, Loss: 2805.0061
Epoch: 04, Loss: 2799.1880
AP SCORE：  0.7419995256533789
AUC SCORE:  0.7656243147230384
Epoch: 00, Loss: 2792.3589
Epoch: 01, Loss: 2792.3208
Epoch: 02, Loss: 2792.2585
Epoch: 03, Loss: 2792.1890
Epoch: 04, Loss: 2792.1245
AP SCORE：  0.7435617154346945
AUC SCORE:  0.7661506074294987
######################### 模块循环 ： 22 ##########################
Epoch: 00, Loss: 2792.0671
Epoch: 01, Loss: 2790.4111
Epoch: 02, Loss: 2787.2825
Epoch: 03, Loss: 2782.8589
Epoch: 04, Loss: 2777.3096
AP SCORE：  0.7466107074992169
AUC SCORE:  0.7665453269593439
Epoch: 00, Loss: 2770.7917
Epoch: 01, Loss: 2770.7551
Epoch: 02, Loss: 2770.6963
Epoch: 03

Epoch: 01, Loss: 2507.5063
Epoch: 02, Loss: 2507.4600
Epoch: 03, Loss: 2507.4077
Epoch: 04, Loss: 2507.3564
AP SCORE：  0.7580107533969217
AUC SCORE:  0.7765887461076268
######################### 模块循环 ： 39 ##########################
Epoch: 00, Loss: 2507.3081
Epoch: 01, Loss: 2506.2705
Epoch: 02, Loss: 2504.3044
Epoch: 03, Loss: 2501.5154
Epoch: 04, Loss: 2498.0061
AP SCORE：  0.7579253448253496
AUC SCORE:  0.7762378843033201
Epoch: 00, Loss: 2493.8787
Epoch: 01, Loss: 2493.8513
Epoch: 02, Loss: 2493.8091
Epoch: 03, Loss: 2493.7671
Epoch: 04, Loss: 2493.7349
AP SCORE：  0.7569898970919446
AUC SCORE:  0.7758870224990132
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 2493.7124
Epoch: 01, Loss: 2492.7070
Epoch: 02, Loss: 2490.8071
Epoch: 03, Loss: 2488.1204
Epoch: 04, Loss: 2484.7468
AP SCORE：  0.7565118271775023
AUC SCORE:  0.775492302969168
Epoch: 00, Loss: 2480.7783
Epoch: 01, Loss: 2480.7539
Epoch: 02, Loss: 2480.7183
Epoch: 03, Loss: 2480.6841
Epoch: 04,

In [9]:
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

print("AUC FIRST MEAN AP : ", np.array(auc_first_best_ap).mean())
print("AUC FIRST STD AP : ", np.array(auc_first_best_ap).std())

print("AUC FIRST MEAN AUC : ", np.array(auc_first_best_auc).mean())
print("AUC FIRST STD AUC : ", np.array(auc_first_best_auc).std())

AP FIRST MEAN AP :  0.7632588275735984
AP FIRST STD AP :  0.01790325575152009
AP FIRST MEAN AUC :  0.7722073593263454
AP FIRST STD AUC :  0.018283290561359678
AUC FIRST MEAN AP :  0.7580107533969217
AUC FIRST STD AP :  0.0
AUC FIRST MEAN AUC :  0.7765887461076268
AUC FIRST STD AUC :  0.0
